# NLP HateSpeech Detection using Transformers (Deep Learninng)

_Deep Learning Project_

----

## Featurization technique Manual 

The one of the tested technique is presented here. The Other technique is with the main notebook.


In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from utility import review_to_words
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


#def data_load(file = 'train_E6oV3lV.csv', test_split: float = 0.1, random_seed: int = 3):
df = pd.read_csv('train_E6oV3lV.csv')
df = df.drop(['id'], axis=1)
X = df['tweet']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
train_X, train_y = shuffle(X_train, y_train)
test_X, test_y = shuffle(X_test, y_test)
#train_X = train_X.apply(lambda x:review_to_words(x))


In [7]:
#print the train and test tweet
print(train_X[1])
print(train_y[1])

@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked
0


In [8]:
#steps for cleaning and tokenization

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [9]:
#try apply for one of the tweets
x = review_to_words(train_X[100])
print(x)

['truli', 'sick', 'ppl']


In [10]:
import pickle
#from utility import review_to_words

def preprocess_data(data_train, data_test, labels_train, labels_test,):
    """Convert each review to words; read from cache if available."""
    words_train = [review_to_words(review) for review in data_train]
    words_test = [review_to_words(review) for review in data_test]
    return words_train, words_test, labels_train, labels_test

In [11]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

In [12]:
#check for the data if preprocessed
train_X[50]

['sad',
 'man',
 'draw',
 'sketch',
 'characterdesign',
 'nerd',
 'geek',
 'friday',
 'flower',
 'dead',
 'caoon']

In [13]:
cache_data = dict(words_train=train_X, words_test=test_X,
                              labels_train=train_y, labels_test=test_y)
with open('preprocessed.pkl', "wb") as f:
    pickle.dump(cache_data, f)

print("Wrote preprocessed data to cache file")

Wrote preprocessed data to cache file


In [14]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for item in data:
        for word in item:
            if word not in word_count:
                word_count[word] = 1
            else:
                word_count[word] += 1
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = sorted(word_count, key = word_count.get, reverse = True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [15]:
# BUild the word dictionary
word_dict = build_dict(train_X)

In [16]:
print(list(word_dict.keys())[0:100])

['user', 'love', 'day', 'happi', 'thank', 'get', 'time', 'life', 'go', 'u', 'like', 'today', 'new', 'posit', 'make', 'father', 'smile', 'see', 'peopl', 'good', 'bihday', 'one', 'want', 'work', 'feel', 'take', 'look', 'friend', 'beauti', 'come', 'need', 'healthi', 'girl', 'wait', 'weekend', 'famili', '2', 'live', 'fun', 'week', 'summer', 'way', 'bull', 'got', 'year', 'think', 'us', 'friday', 'great', 'best', 'music', 'first', 'follow', 'sta', 'know', 'morn', '3', 'back', 'say', 'world', 'watch', 'realli', 'sad', 'thing', 'home', 'dad', 'ur', 'never', 'tomorrow', 'final', 'cute', 'even', '1', 'affirm', 'blog', 'right', 'orlando', 'night', 'model', 'trump', 'sunday', 'fathersday', 'well', 'much', 'last', 'play', 'find', 'bless', 'let', 'hope', 'show', 'next', 'alway', 'amaz', '4', 'selfi', 'guy', 'excit', 'still', 'quot']


In [17]:
with open('word_dict.pkl', "wb") as f:
    pickle.dump(word_dict, f)

print("Wrote preprocessed data to pickle file")

Wrote preprocessed data to pickle file


In [18]:
def convert_and_pad(word_dict, sentence, pad=256):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=256):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [19]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [20]:
print(train_X[100])
print(train_X_len[100])

[  62 2542   20    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [22]:
import pandas as pd
import os
    
df_pro = pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join('train.csv'), header=False, index=False)